In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/GaParmar/img2img-turbo.git

Cloning into 'img2img-turbo'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 255 (delta 88), reused 57 (delta 56), pack-reused 139 (from 2)
Receiving objects: 100% (255/255), 31.91 MiB | 53.65 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [ ]:
import os
os.chdir('img2img-turbo/')

In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-cr4n_cfe/clip_7e037eb05f404e9fae27941bd6db1712
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-cr4n_cfe/clip_7e037eb05f404e9fae27941bd6db1712
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s

In [ ]:
# Pointing to dataset loc
import os, pathlib, shutil

drive_data_root = "/content/drive/MyDrive/cycleGAN_dataset_test"  # folder matching required training folder structure (train_A, train_B, test_A, test_B + fixed prompts)
dataset_name = "my_sat2uav"

repo_root = "/content/img2img-turbo"
data_dst = f"{repo_root}/data/{dataset_name}"
os.makedirs(f"{repo_root}/data", exist_ok=True)

# Since dataset is already organised as specified, symlink easier to implement.
if os.path.exists(data_dst):
    shutil.rmtree(data_dst)
os.symlink(drive_data_root, data_dst, target_is_directory=True)

# Double checking file listing
for split in ("train_A","train_B","test_A","test_B"):
    print(split, "=>", len(list(pathlib.Path(data_dst, split).glob("*"))), "files")


train_A => 50 files
train_B => 29 files
test_A => 5 files
test_B => 3 files


In [ ]:
# Creating/modifying the required fixed prompts dataset (as per img2img training doc).
open(f"{data_dst}/fixed_prompt_a.txt","w").write("overhead satellite scene")
open(f"{data_dst}/fixed_prompt_b.txt","w").write("overhead uav scene")

18

In [ ]:
!pwd && ls

/content/img2img-turbo
assets	environment.yaml	LICENSE		  scripts
data	gradio_canny2image.py	README.md	  src
docs	gradio_sketch2image.py	requirements.txt  style.css


In [ ]:
!env

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=f9ff959eac1d
MODEL_PROXY_HOST=https://mp.kaggle.net
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,

In [ ]:
 !nvidia-smi

Thu Oct 30 14:09:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!accelerate config

--------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
0
This machine
--------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-HPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-SDAA
    multi-MUSA
    TPU
0
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:NO
Do you want to use DeepSpeed? [yes/NO]: NO
What GPU(s) (by id) should be used for training on this machine as a comma-separated list? [all]:0
Would you like to enable numa efficiency? (Currently only support

In [ ]:
%%bash
export NCCL_P2P_DISABLE=1
export TF_CPP_MIN_LOG_LEVEL=3        # optional, silences TensorFlow spam
export WANDB_MODE=offline            # ✅ disables online sync
export WANDB_PROJECT=horse2zebra     # any name is fine
export WANDB_DISABLE_CODE=true       # optional (avoids uploading code)
accelerate launch --main_process_port 29501 src/train_cyclegan_turbo.py \
    --pretrained_model_name_or_path="stabilityai/sd-turbo" \
    --output_dir="/content/drive/MyDrive/outputs/cycleGAN_test" \
    --dataset_folder "/content/drive/MyDrive/cycleGAN_dataset_test" \
    --train_img_prep "resize_286_randomcrop_256x256_hflip" --val_img_prep "no_resize" \
    --learning_rate="1e-5" --max_train_steps=5000 \
    --train_batch_size=1 --gradient_accumulation_steps=1 \
    --report_to "wandb" --tracker_project_name "gparmar_unpaired_h2z_cycle_debug_v2" \
    --enable_xformers_memory_efficient_attention --validation_steps 250 \
    --lambda_gan 0.5 --lambda_idt 1 --lambda_cycle 1

Found 3 images in the folder /content/drive/MyDrive/outputs/cycleGAN_test/fid_reference_a2b
Found 5 images in the folder /content/drive/MyDrive/outputs/cycleGAN_test/fid_reference_b2a
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_vitbase8_pretrain.pth
Found 5 images in the folder /content/drive/MyDrive/outputs/cycleGAN_test/fid-1/samples_a2b
step=1, fid(a2b)=490.50, dino(a2b)=0.391
Found 3 images in the folder /content/drive/MyDrive/outputs/cycleGAN_test/fid-1/samples_b2a
step=1, fid(b2a)=562.2590942145745, 

E0000 00:00:1761843090.730430    2496 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761843090.737093    2496 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761843090.755023    2496 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761843090.755053    2496 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761843090.755056    2496 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761843090.755058    2496 computation_placer.cc:177] computation placer already registered. Please check linka

Inference

In [ ]:
%%bash
python src/inference_unpaired.py --model_path "output/cyclegan_turbo/my_horse2zebra/checkpoints/model_1001.pkl" \
    --input_image "data/my_horse2zebra/test_A/n02381460_20.jpg" \
    --prompt "picture of a zebra" --direction "a2b" \
    --output_dir "outputs" --image_prep "no_resize"